In [8]:
import pandas as pd
import plotly 
import plotly.express as px
import dash
import dash_table
import dash_core_components as dcc
from dash.dependencies import Input,Output 
import dash_html_components as html

In [2]:
app=dash.Dash(__name__)

In [3]:
df=pd.read_excel(r"C:\Users\Admin\OneDrive - Brandscapes Consultancy Pvt. Ltd\Desktop\datasets\COVID-19-geographic-disbtribution-worldwide-2020-03-29.xlsx")

In [4]:
df.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
0,2020-03-29,29,3,2020,15,1,Afghanistan,AF,AFG,37172386.0
1,2020-03-28,28,3,2020,16,1,Afghanistan,AF,AFG,37172386.0
2,2020-03-27,27,3,2020,0,0,Afghanistan,AF,AFG,37172386.0
3,2020-03-26,26,3,2020,33,0,Afghanistan,AF,AFG,37172386.0
4,2020-03-25,25,3,2020,2,0,Afghanistan,AF,AFG,37172386.0


In [5]:
dff=df.groupby('countriesAndTerritories',as_index=False)[['deaths','cases']].sum()

In [6]:
dff.head()

,countriesAndTerritories,deaths,cases
0,Afghanistan,3,106
1,Albania,10,197
2,Algeria,26,409
3,Andorra,4,308
4,Angola,0,4


In [11]:
app.layout = html.Div([
    html.Div([
        dash_table.DataTable(
            id='datatable_id',
            data=dff.to_dict('records'),
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": False} for i in dff.columns
            ],
            editable=False,
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            row_selectable="multi",
            row_deletable=False,
            selected_rows=[],
            page_action="native",
            page_current= 0,
            page_size= 6,
            
            style_cell_conditional=[
                {'if': {'column_id': 'countriesAndTerritories'},
                 'width': '40%', 'textAlign': 'left'},
                {'if': {'column_id': 'deaths'},
                 'width': '30%', 'textAlign': 'left'},
                {'if': {'column_id': 'cases'},
                 'width': '30%', 'textAlign': 'left'},
            ],
        ),
    ],className='row'),

    html.Div([
        html.Div([
            dcc.Dropdown(id='linedropdown',
                options=[
                         {'label': 'Deaths', 'value': 'deaths'},
                         {'label': 'Cases', 'value': 'cases'}
                ],
                value='deaths',
                multi=False,
                clearable=False
            ),
        ],className='six columns'),

        html.Div([
        dcc.Dropdown(id='piedropdown',
            options=[
                     {'label': 'Deaths', 'value': 'deaths'},
                     {'label': 'Cases', 'value': 'cases'}
            ],
            value='cases',
            multi=False,
            clearable=False
        ),
        ],className='six columns'),

    ],className='row'),

    html.Div([
        html.Div([
            dcc.Graph(id='linechart'),
        ],className='six columns'),

        html.Div([
            dcc.Graph(id='piechart'),
        ],className='six columns'),

    ],className='row'),


])

In [12]:
@app.callback(
    [Output('piechart', 'figure'),
     Output('linechart', 'figure')],
    [Input('datatable_id', 'selected_rows'),
     Input('piedropdown', 'value'),
     Input('linedropdown', 'value')]
)
def update_data(chosen_rows,piedropval,linedropval):
    if len(chosen_rows)==0:
        df_filterd = dff[dff['countriesAndTerritories'].isin(['China','Iran','Spain','Italy'])]
    else:
        print(chosen_rows)
        df_filterd = dff[dff.index.isin(chosen_rows)]

    pie_chart=px.pie(
            data_frame=df_filterd,
            names='countriesAndTerritories',
            values=piedropval,
            hole=.3,
            labels={'countriesAndTerritories':'Countries'}
            )


    
    list_chosen_countries=df_filterd['countriesAndTerritories'].tolist()
   
    df_line = df[df['countriesAndTerritories'].isin(list_chosen_countries)]

    line_chart = px.line(
            data_frame=df_line,
            x='dateRep',
            y=linedropval,
            color='countriesAndTerritories',
            labels={'countriesAndTerritories':'Countries', 'dateRep':'date'},
            )
    line_chart.update_layout(uirevision='foo')

    return (pie_chart,line_chart)


In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
[0]
[0]
[0]
[0, 1]
